# Chapter 19: Training and Deploying TensorFlow Models at Scale

## 1. Pendahuluan

Pada bab ini, kita akan mempelajari bagaimana cara melatih dan mendistribusikan model TensorFlow pada skala besar. Fokusnya adalah memanfaatkan infrastruktur komputasi modern (multi-GPU, multi-node, dan Cloud) agar pelatihan lebih cepat dan deployment lebih andal.


## 2. Mengapa Butuh Skala Besar

Beberapa alasan model ML memerlukan pelatihan skala besar:
- Dataset sangat besar (misalnya ImageNet, video streaming)
- Model sangat kompleks (Transformer, LLM)
- Waktu pelatihan perlu dipercepat (reduksi waktu dari minggu ke jam)


## 3. Strategi Distribusi di TensorFlow

TensorFlow menyediakan modul `tf.distribute` untuk mempermudah paralelisme.

Strategi distribusi umum:
- MirroredStrategy: Distribusi pada multi-GPU dalam satu mesin.
- MultiWorkerMirroredStrategy: Distribusi pada beberapa worker di cluster.
- TPUStrategy: Optimasi untuk TPU di cloud.
- ParameterServerStrategy: Untuk cluster besar dengan parameter server.


## 4. Contoh Penggunaan MirroredStrategy

Contoh multi-GPU di satu mesin:


In [ ]:
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


## 5. MultiWorkerMirroredStrategy

Untuk skenario multi-node:

1. Jalankan environment variable TF_CONFIG di tiap node:

```json
{
  "cluster": {
    "worker": [
      "worker0.example.com:12345",
      "worker1.example.com:12345"
    ]
  },
  "task": {"type": "worker", "index": 0}
}




2. Gunakan script Python dengan MultiWorkerMirroredStrategy.


## 6. Sinkronisasi Gradien

Dalam training distributed, setiap replica menghitung gradien. TensorFlow otomatis melakukan sinkronisasi gradien antar device untuk memastikan parameter tetap konsisten.


## 7. Menangani Dataset Besar

Gunakan `tf.data` untuk pipeline input efisien:
- `shuffle` untuk acak data.
- `prefetch` untuk overlap I/O.
- `interleave` untuk membaca banyak file paralel.

Contoh:


In [ ]:
import tensorflow as tf

dataset = tf.data.TFRecordDataset(["data1.tfrecord", "data2.tfrecord"])
dataset = dataset.shuffle(1000).repeat().batch(32).prefetch(tf.data.AUTOTUNE)


## 8. Training di Cloud

Provider cloud populer:
- Google Cloud AI Platform
- AWS SageMaker
- Azure ML

Keuntungan:
- TPU atau GPU high-end tersedia.
- Autoscaling.
- Monitoring dan logging terintegrasi.


## 9. Export Model untuk Serving

Gunakan format SavedModel agar model siap deploy ke TensorFlow Serving atau cloud.


In [ ]:
model.save("my_saved_model")

import tensorflow as tf
model = tf.keras.models.load_model("my_saved_model")


## 10. TensorFlow Serving

TensorFlow Serving adalah server high-performance untuk melayani model ML.

Contoh command Docker:

```bash
docker run -p 8501:8501 \
  --mount type=bind,source=$(pwd)/my_saved_model,target=/models/my_model \
  -e MODEL_NAME=my_model -t tensorflow/serving

```
API:

* REST: http://localhost:8501/v1/models/my_model:predict

## 11. Monitoring dan A/B Testing

Untuk production, penting untuk:
- Monitor performa model (latency, throughput, error rate)
- A/B Testing untuk membandingkan versi model
- Canary Deployment: rilis model baru ke subset user untuk uji stabilitas.
